In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

In [2]:
import pickle
import sys
sys.path.insert(0, '../../../summarisation/preprocess')
import vectorizer

from random import shuffle

df, vecs = pickle.load(open('../../../summarisation/preprocess/pico_vecdf_mindf10.p', 'rb'))

lang_1 = vecs['abstract']
lang_2 = vecs['title']
SOS_token = lang_1.word2idx['<SOS>']
EOS_token = lang_1.word2idx['<EOS>']

sentences = pickle.load(open('../../../summarisation/preprocess/abstract_sentences.p', 'rb'))
lang_1.sents = [sent for sent in sentences if len(sent) < 100 and len(sent) > 3]
shuffle(lang_1.sents)

/home/sarthak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[]


In [3]:
len(lang_1.sents)

464228

In [9]:
data = {'word2idx' : lang_1.word2idx, 'idx2word' : lang_1.idx2word, 
        'training' : lang_1.sents, 'embeddings' : lang_1.embeddings}

In [10]:
#pickle.dump(data, open('data_pico.p', 'wb'))

In [3]:
from nltk.corpus import stopwords

In [4]:
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr = WordNetLemmatizer()

In [5]:
lemss = []
for word in lang_1.word2idx :
    lemss.append(lmtzr.lemmatize(word))

In [6]:
new_sents = []
import re
stops = stopwords.words('english')
for sent in lang_1.sents :
    sent = [lang_1.idx2word[idx] for idx in sent]
    sent = [re.sub('[^a-zA-Z0-9]', '', x).strip() for x in sent]
    sent = [x for x in sent if len(x) > 0]
    sent = [word for word in sent if word not in stops]
    sent = [lmtzr.lemmatize(word) for word in sent]
    new_sents.append(sent)

In [8]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('../../wikipedia-pubmed-and-PMC-w2v.bin', binary=True)

In [9]:
from collections import defaultdict
training_data = new_sents
vocab = defaultdict(int)
for line in training_data :
    for word in line :
        vocab[word] += 1

In [10]:
word2idx = {'[0]' : 0}
idx = len(word2idx)
for word in vocab :
    word2idx[word] = idx
    idx += 1

In [11]:
idx_train = []
for i, line in enumerate(training_data) :
    nline = []
    for word in line :
        if any(char.isdigit() for char in word) :
            nline.append(word2idx['qqq'])
        elif word in word2idx :
            nline.append(word2idx[word])
        else :
            nline.append(word2idx['unk'])
    idx_train.append(nline)
    if i%100000 == 0:
        print(i)

0
100000
200000
300000
400000


In [12]:
idx2word = {v:k for k,v in word2idx.items()}
data = {'word2idx' : word2idx, 'idx2word' : idx2word, 'training' : idx_train}

In [13]:
import numpy as np
embeddings = np.zeros((len(data['word2idx']), 200))

In [14]:
for word in data['word2idx'] :
    if word in model.vocab :
        embeddings[data['word2idx'][word], :] = list(model[word])
    else :
        embeddings[data['word2idx'][word], :] = np.random.randn(200)

In [15]:
data['embeddings'] = embeddings
pickle.dump(data, open('data_pico_new.p', 'wb'))